# Item 33: Compose Multiple Generators with `yield from`

Generators are so useful that many programs start lo look like layers of generators strung together.

In [1]:
# Say that we have a graphical program that's using generators to animate the movements of images onscreen
def move(period, speed):
    for _ in range(period):
        yield speed

def pause(delay):
    for _ in range(delay):
        yield 0

In [2]:
# To create the final animation, we need to combine move and pause to produce a single sequence of onscreen deltas. 
# Here, we do this by calling a generator for each step of the animation, iterating over each generator in turn, and
# yielding the delta of all from them in sequence
def animate():
    for delta in move(4, 5.0):
        yield delta
    for delta in pause(3):
        yield delta
    for delta in move(2, 3.0):
        yield delta

In [3]:
# Now we can render those deltas onscreen as they're produced by the single animation generator
def render(delta):
    print(f'Delta: {delta:.1f}')
    # Move the images onscreen
    # ...

def run(func):
    for delta in func():
        render(delta)

run(animate)

Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 3.0
Delta: 3.0


The problem with the code above is the repetitive nature of the animate function. The redundancy of the `for` statements and `yield` expressions for each generator adds noise and reduces readability. This example only includes three nested generators and it's already hurting clarity; a complex animation with a dozen or more would be extremely difficult to follow.

The solution to this proble is to use the `yield from` expression. This advanced generator feature allows us to yield all values from a nested generator before returning control to the parent generator. 

In [4]:
# Here, we implement the animation function by using yield from
def animate_composed():
    yield from move(4, 5.0)
    yield from pause(3)
    yield from move(2, 3.0)

run(animate_composed)

Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 3.0
Delta: 3.0


As we can see, the result is the same as before, but now the code is clearer and more intuitive. `yield from` essentially causes the Python interpreter to handle the nested `for` loop and yield expression boilerplate for you, which results in better performance.

In [5]:
# Here, we verify the speedup by using the timeit built-in module to run a micro-benchmark
import timeit

def child():
    for i in range(1_000_000):
        yield i
    
def slow():
    for i in child():
        yield i

def fast():
    yield from child()

baseline = timeit.timeit(
    stmt='for _ in slow(): pass',
    globals=globals(),
    number=50
)
print(f'Manual nesting {baseline:.2f}s')

comparison = timeit.timeit(
    stmt='for _ in fast(): pass',
    globals=globals(),
    number=50
)
print(f'Composed nesting {comparison:.2f}s')

reduction = -(comparison - baseline) / baseline
print(f'{reduction:.1%} less time')

Manual nesting 3.61s
Composed nesting 3.13s
13.2% less time


The author strongly recommends that, if we find ourselves composing generators, we should use `yield from` when possible.